<a href="https://colab.research.google.com/github/Geeteshkamble/AI/blob/main/nb/Llama3.1_(8B)-GRPO%2002/05/2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [3]:
import unsloth
import pandas as pd
from unsloth import FastModel
import torch
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template, standardize_data_formats, train_on_responses_only
from transformers import TextStreamer
import os
import sys
import re
import requests

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-02 08:58:53 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-02 08:58:53 [__init__.py:239] Automatically detected platform cuda.


In [4]:
def load_and_prepare_dataset(csv_path):
    """Load and prepare the dataset."""
    try:
        df = pd.read_csv(csv_path, encoding='utf-8', names=['Child', 'Robot'])
        conversations = []
        for _, row in df.iterrows():
            if pd.isna(row['Child']) or pd.isna(row['Robot']) or row['Child'] == 'Child':
                continue
            child_text = str(row['Child']).strip()
            robot_text = str(row['Robot']).strip()
            if child_text and robot_text:
                conversation = [
                    {"role": "user", "content": [{"type": "text", "text": child_text}]},
                    {"role": "assistant", "content": [{"type": "text", "text": robot_text}]}
                ]
                conversations.append({"conversations": conversation})
        if not conversations:
            raise ValueError("No valid conversations found in the dataset")
        dataset = Dataset.from_list(conversations)
        return dataset
    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        raise

In [5]:
def format_dataset(dataset, tokenizer):
    """Format the dataset using the chat template."""
    def formatting_prompts_func(examples):
        convos = examples["conversations"]
        texts = []
        for convo in convos:
            formatted_text = ""
            for message in convo:
                role = message["role"]
                content = message["content"][0]["text"]
                if role == "user":
                    formatted_text += f"<start_of_turn>user\n{content}<end_of_turn>\n"
                else:
                    formatted_text += f"<start_of_turn>model\n{content}<end_of_turn>\n"
            texts.append(formatted_text)
        return {"text": texts}
    return dataset.map(formatting_prompts_func, batched=True)

In [6]:
def train_model(dataset, model_name="meta-llama/meta-Llama-3.1-8B-Instruct", max_steps=60):
    """Train the model."""
    model, tokenizer = FastModel.from_pretrained(
        model_name=model_name,
        max_seq_length=2048,
        load_in_4bit=True,
        load_in_8bit=False,
        full_finetuning=False,
    )
    model = FastModel.get_peft_model(
        model,
        finetune_vision_layers=False,
        finetune_language_layers=True,
        finetune_attention_modules=True,
        finetune_mlp_modules=True,
        r=8,
        lora_alpha=8,
        lora_dropout=0,
        bias="none",
        random_state=3407,
    )
    dataset = format_dataset(dataset, tokenizer)
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        eval_dataset=None,
        args=SFTConfig(
            dataset_text_field="text",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=5,
            max_steps=max_steps,
            learning_rate=2e-4,
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            report_to="none",
            dataset_num_proc=2,
        ),
    )
    trainer_stats = trainer.train()
    return model, tokenizer, trainer_stats

In [7]:
def test_model(model, tokenizer, test_inputs):
    """Test the trained model with sample inputs."""
    results = []
    for input_text in test_inputs:
        messages = [{
            "role": "user",
            "content": [{"type": "text", "text": input_text}]
        }]
        text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
        )
        outputs = model.generate(
            **tokenizer([text], return_tensors="pt").to("cuda"),
            max_new_tokens=200,
            temperature=1.0,
            top_p=0.95,
            top_k=64,
        )
        response = tokenizer.batch_decode(outputs)[0]
        results.append({"input": input_text, "response": response})
    return results


In [ ]:
def main():
    print("Loading and preparing dataset...")
    dataset = load_and_prepare_dataset("/content/Negative words.csv")
    print("\nStarting model training...")
    model, tokenizer, trainer_stats = train_model(dataset)
    print(f"\nTraining completed in {trainer_stats.metrics['train_runtime']} seconds")
    print("\nTesting model with sample inputs:")
    test_inputs = [
        "I don't want to do my homework!",
        "I don't like vegetables!",
        "I don't want to go to school today!",
    ]
    results = test_model(model, tokenizer, test_inputs)
    for result in results:
        print(f"\nInput: {result['input']}")
        print(f"Response: {result['response']}")

if __name__ == "__main__":
    main()